## Writing a Binary Cmdlet
As described previously, cmdlets are light weight commands that are part of the PowerShell environment. Cmdlets can be written as .NET classes in C#, or as PowerShell scripts.
Script based cmdlets are explained in the [Functions](6.%20Functions.ipynb) notebook. This notebook describes how to write a rudimentary binary cmdlet using C#.

In the PowerShell code below, a the .NET class is written that describes a new cmdlet called `Get-CodePids`. The cmdlet gets the PIDs of all running VSCode processes. The class is stored in a variable called `$source`, and then the `Add-Type` cmdlet is called to compile `$source` into a temporary executable file known in .NET as an _assembly_.

The `Import-Module` cmdlet can take in an assembly and load it into the current PowerShell runtime as a _module_, making the commands contained in the assembly available for use. Calling the `Get-CodePids` cmdlet should print the aforementioned PIDs.

The rest of the C# code below is explained with comments, however a full explanation is outside the scope of this notebook.

In [1]:
$source = @"
using System.Management.Automation;
using System.Diagnostics;

// a cmdlet class needs to be decorated with the Cmdlet attribute
// the verbscommon option decides which PowerShell verd to use and the cmdlet name
[Cmdlet(VerbsCommon.Get, "CodePids")]

// the Cmdlet class contained in the System.Management.Automation namespace is the base class that
// the cmdlet code we are writing inherits from
public class GetCodePids : Cmdlet {

    // this line gets all the VSCode processes and stores them in an array
    Process[] allCode = Process.GetProcessesByName("code");

    // the Cmdlet base class contains methods for the Begin, Process and End blocks we 
    // have already seen in PowerShell functions - at least the ProcessRecord() method must be
    // overridden in the new cmdlet class 
    protected override void ProcessRecord() {
        foreach(Process p in allCode) {
            
            // cmdlets emit .NET objects that are passed along the pipeline to other
            // cmdlets, this is handled by the WriteObject() method 
            WriteObject("PID: " + p.Id + ", Started: " + p.StartTime);

            // note - usually cmdlets dont format output like this, and instead lets 
            // other cmdlets on the pipeline format what gets printed, but for this demonstration
            // this cmdlet is returning a formatted string object
        }
    }
}
"@

Import-Module -Assembly (Add-Type -TypeDefinition $source -PassThru).Assembly
Get-CodePids

PID: 9756, Started: 5/30/2024 4:36:50 PM
PID: 2548, Started: 5/30/2024 4:36:50 PM
PID: 8568, Started: 5/30/2024 4:36:50 PM
PID: 15768, Started: 5/30/2024 4:36:50 PM
PID: 10032, Started: 5/30/2024 4:36:51 PM
PID: 416, Started: 5/30/2024 4:36:52 PM
PID: 16332, Started: 5/30/2024 4:36:53 PM
PID: 10408, Started: 5/30/2024 4:36:53 PM
PID: 14640, Started: 5/30/2024 4:36:53 PM
PID: 3748, Started: 5/30/2024 4:36:56 PM
